<a href="https://colab.research.google.com/github/negarhonarvar/UT-AI-Hackathon/blob/main/FootballerClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Footballer Classification

In this task, we classify footballer robots into 4 different categories:

*   Defender
*   Forward
*   Goalkeeper
*   Midfielder



## Libraries

In [ ]:
import os
import csv
import math
import torch
import random
import shutil
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from PIL import Image

## Drive Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

TRAIN_DIR = "/content/drive/MyDrive/HeatMap_Data/train"
TEST_DIR = "/content/drive/MyDrive/HeatMap_Data/test"
SUBMISSION_FILE = "/content/drive/MyDrive/HeatMap_Data/submission.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Parameter Set Up

In [ ]:
NUM_CLASSES = 4
BATCH_SIZE = 16
NUM_EPOCHS = 30
LEARNING_RATE = 1e-4
VAL_SPLIT = 0.1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Data Augmentation

In [ ]:
train_transforms = T.Compose([
    T.RandomResizedCrop(224, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(15),
    T.RandomAffine(degrees=0, translate=(0.1, 0.1), shear=10),
    T.RandomPerspective(distortion_scale=0.2, p=0.5),
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

val_transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

## Data Loaders

In [ ]:
def get_train_val_loaders():
    dataset = torchvision.datasets.ImageFolder(root=TRAIN_DIR, transform=train_transforms)
    num_data = len(dataset)
    val_size = int(VAL_SPLIT * num_data)
    train_size = num_data - val_size
    train_ds, val_ds = random_split(dataset, [train_size, val_size])
    val_ds.dataset.transform = val_transforms

    targets = [dataset.targets[i] for i in train_ds.indices]
    class_counts = np.bincount(targets)
    class_weights = 1. / class_counts
    sample_weights = [class_weights[t] for t in targets]
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(train_ds), replacement=True)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    return train_loader, val_loader, dataset

## Model

In [ ]:
def create_model(num_classes=NUM_CLASSES):
    model = torchvision.models.resnet50(pretrained=True)
    for param in model.parameters():
        param.requires_grad = True
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

In [ ]:
def validate_model(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
def train_model(model, train_loader, val_loader, epochs, lr):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)
    model.to(DEVICE)
    best_acc = 0.0
    best_model_wts = None

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_samples = 0
        correct_samples = 0

        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_samples += (preds == labels).sum().item()
            total_samples += labels.size(0)

        train_loss = running_loss / total_samples
        train_acc = correct_samples / total_samples

        val_loss, val_acc = validate_model(model, val_loader, criterion)
        scheduler.step(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = {k: v.cpu() for k, v in model.state_dict().items()}

        print(f"Epoch {epoch+1}/{epochs}: Train loss {train_loss:.4f}, Train acc {train_acc*100:.2f}%, Val loss {val_loss:.4f}, Val acc {val_acc*100:.2f}%")

    if best_model_wts is not None:
        model.load_state_dict({k: v.to(DEVICE) for k, v in best_model_wts.items()})
    return model


In [ ]:
def predict_test_images(model, dataset):
    model.eval()
    idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}
    test_files = sorted(os.listdir(TEST_DIR), key=lambda x: int(os.path.splitext(x)[0]))
    predictions = []
    with torch.no_grad():
        for file in test_files:
            path = os.path.join(TEST_DIR, file)
            pil_img = Image.open(path).convert("RGB")
            tensor_img = val_transforms(pil_img).unsqueeze(0).to(DEVICE)
            outputs = model(tensor_img)
            _, predicted_idx = torch.max(outputs, 1)
            predicted_class = idx_to_class[predicted_idx.item()]
            predictions.append(predicted_class)
    return predictions

## Submission Generation

In [ ]:
def write_submission(predictions):
    with open(SUBMISSION_FILE, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Position"])
        for pred in predictions:
            writer.writerow([pred])

## Main

In [ ]:
def main():
    train_loader, val_loader, dataset = get_train_val_loaders()
    print(f"Training samples: {len(train_loader.dataset)}, Validation samples: {len(val_loader.dataset)}")
    model = create_model(num_classes=NUM_CLASSES)
    model = train_model(model, train_loader, val_loader, NUM_EPOCHS, LEARNING_RATE)
    predictions = predict_test_images(model, dataset)
    write_submission(predictions)
    print(f"Submission saved to {SUBMISSION_FILE}")

In [ ]:
if __name__ == "__main__":
    main()

Training samples: 358, Validation samples: 39


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 190MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/30: Train loss 0.4263, Train acc 89.66%, Val loss 0.1121, Val acc 97.44%
Epoch 2/30: Train loss 0.1282, Train acc 95.25%, Val loss 0.0416, Val acc 100.00%
Epoch 3/30: Train loss 0.1050, Train acc 96.37%, Val loss 0.1128, Val acc 97.44%
Epoch 4/30: Train loss 0.0862, Train acc 97.49%, Val loss 0.0495, Val acc 97.44%
Epoch 5/30: Train loss 0.0494, Train acc 98.32%, Val loss 0.0287, Val acc 100.00%
Epoch 6/30: Train loss 0.0182, Train acc 99.72%, Val loss 0.0899, Val acc 94.87%
Epoch 7/30: Train loss 0.0083, Train acc 99.72%, Val loss 0.0306, Val acc 100.00%
Epoch 8/30: Train loss 0.0070, Train acc 100.00%, Val loss 0.0192, Val acc 100.00%
Epoch 9/30: Train loss 0.0074, Train acc 99.72%, Val loss 0.0136, Val acc 100.00%
Epoch 10/30: Train loss 0.0042, Train acc 100.00%, Val loss 0.0145, Val acc 100.00%
Epoch 11/30: Train loss 0.0022, Train acc 100.00%, Val loss 0.0226, Val acc 100.00%
Epoch 12/30: Train loss 0.0117, Train acc 99.44%, Val loss 0.0244, Val acc 100.00%
Epoch 13/30: T